# Reinforcement Learning with Human Feedback (RLHF)

* https://gist.github.com/JoaoLages/c6f2dfd13d2484aa8bb0b2d567fbf093
* https://huggingface.co/blog/rlhf
* https://medium.com/mantisnlp/finetuning-an-llm-rlhf-and-alternatives-part-ii-8c1846910625

# 1 - Introduction

Unfortunately, Supervised fine-tuning (SFT) is often insufficient for refining the large language models to meet specific  requirements.

With it, you can complement the lack of information you may have in pretrained LLM, change the style, structure, teach to answer differently to different outputs.

However, **there will always be multiple ways you can answer a question**. Conversational models, such as ChatGPT, have been traditionally requiring an additional step called Reinforcement Learning by Human Feedback (RLHF).

Reinforcement learning is a paradigm in which an agent learns to make decisions by receiving feedback from its environment. In the context of language models, this feedback is provided by human reviewers who assess and rate the model's responses. By leveraging human expertise and judgements, reinforcement learning facilitates the iterative improvement of the model's performance and fine-tunes its responses.

The process of reinforcement learning by human feedback involves several important steps:

1. Guidelines are defined to guarantee unique criteria when deciding what is a good and a bad answer to an input.
2. A Reward Model (RM) should be trained, which will evaluate each of the responses in terms of accuracy, relevance, and adherence to guidelines.
3. To train the RM, some prompts are selected and sent to human reviewers. We call them Preference data (PD).
4. The reviewers then interact with the model and manually evaluate and rate the corresponding outputs.
5. The collected feedback, in the form of ratings or rankings, is used to train the reward model.
6. With the RM trained, we can train a Policy Optimizer, a required component which will guide the fine-tuning of the LLM.
7. We fine-tune the LLM with Policy Optimization.
8. This iterative feedback loop allows the model to gradually learn from human guidance and refine its behaviour accordingly.

# 2 - Preference data

The main requirement at this point is having preference data. Preference data is a collection of options / alternatives to a prompt, which can be sent to a group of Annotators and/or Subject matter experts (SME) so they can rate it, give some annotation guidelines, from the best to the worst.

Several approaches have been used to create the Preference data:

1. For selecting the best prompts, you can either have a predefined set of them, have a predefined template and generate some on the fly, etc.
2. For selecting the answers, you can send the prompt to one specific version of the model (e.g., the last one you fine-tuned) or to different checkpoints. 
3. Decide how many answers you want to rank: you can use two and have a binary ranking schema or have your annotators ranking from 1 to 5, as an example.

> Always define first the annotation guidelines for the different ranking levels, to minimize individual interpretations and standarize the answers.

There are also several datasets you can use, which you can find in this GitHub repository:

| Dataset Name                                           | Type              | Size   | Description                                                                                                                                                                     |
|-------------------------------------------------------|-------------------|--------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| OpenAI WebGPT Comparisons                             | Question-Answering | 20k    | 20k comparisons where each example comprises a question, a pair of model answers, and human-rated preference scores for each answer. RLHF dataset used to train the OpenAI WebGPT reward model. |
| OpenAI Summarization                                   | Summarization      | 64k    | 64k text summarization examples including human-written responses and human-rated model responses. RLHF dataset used in the OpenAI Learning to Summarize from Human Feedback paper. |
| OpenAssistant Conversations Dataset (OASST1)         | Conversations      | 461k   | A human-generated, human-annotated assistant-style conversation corpus consisting of 161k messages in 35 languages, annotated with 461k quality ratings, resulting in 10k+ fully annotated conversation trees. |
| Stanford Human Preferences Dataset (SHP)              | Question-Answering | 385k   | 385K collective human preferences over responses to questions/instructions in 18 domains for training RLHF reward models and NLG evaluation models.                            |
| Reddit ELI5                                           | Question-Answering | 270k   | 270k examples of questions, answers, and scores collected from 3 Q&A subreddits.                                                                                             |
| Human ChatGPT Comparison Corpus (HC3)                | Question-Answering | 60k    | 60k human answers and 27K ChatGPT answers for around 24K questions.                                                                                                          |

This is what the [OpenAI Summarization Preference data](https://huggingface.co/datasets/openai/summarize_from_feedback) looks like. In the example we have a summarization task (left) and one possible answer (right) with a ranking/score (in this case we have several metrics like accuracy, coverage, etc., being the final aggregated metric called "overall").

<table>
    <tr>
        <td><img src="./images_1/preference_data_example.png" width="900"/></td>
    </tr>
</table>

[You can generate a preference dataset using the Arguilla tool](https://argilla.io/)

<table>
    <tr>
        <td><img src="./images_1/arguilla.png" width="400"/></td>
    </tr>
</table>

## 2.1 - Instruction data for fine-tuning vs preference data for reinforcement learning

So far we have seen we need 2 datasets to get a conversational LLM working:
* Instruction data for fine-tuning
* Preference data for reinforcement learning

Let's clarify the differences:

The **instruction data** is in the fine-tuning step, and consists of prompts and expected answers which aim to improve or correct the way an LLM generates text based answers based on a prompt. You may even want to train the model on a generation of a new type of content. For example, a piece of advertisement SEO content, a report, etc. The data consists of rows with 1 prompt and 1 generated example to it.

The **preference data** is used to train a policy. Basically, here you are not teaching the model anymore to write specific emails, or reports, or any content of your needs. You are training a Policy model to make the model distinguish, given $N$ acceptable answers to a prompt, which are better than others. Usually preference data is a collection of 1 prompt and $N$ answers to that prompt, to then be ranked fro a group of Annotators or Subject Matter Experts.

This is how Chat-GPT depicts it:

<table>
    <tr>
        <td><img src="./images_1/chat_gpt_data.png" width="700"/></td>
    </tr>
</table>

# 3 - The reward model and the policy optimizer

LHF adds a big complexity to the process of training your custom LLM. The following picture summarizes everything you need to do:

<table>
    <tr>
        <td><img src="./images_1/rlhf_summary.png" width="700"/></td>
    </tr>
</table>



## 3.1 - Reward model training

Generating a reward model (RM, also referred to as a preference model) calibrated with human preferences is where the relatively new research in RLHF begins. The underying goal is to get a model or system that takes in a sequence of text, and returns a scalar reward which should numerically represent the human preference. The system can be an end-to-end LM, or a modular system outputting a reward (e.g., a model ranks outputs, and then ranking is converted to reward). The output being a scalar reward is crucial for existing RL algorithms being integrated seamlessly later in the RLHF process.

These LMs for reward modeling can be both another fine-tuned LM or a LM trained from scratch on the preference data. For example, Anthropic has used a specialized method of fine-tuning to initialize these models after pretraining (preference model pretraining, PMP) because they found it to be more sample efficient than fine-tuning, but no one base model is considered the clear best choice for reward models.

The training dataset of prompt-generation pairs for the RM is generated by sampling a set of prompts from a predefined dataset.

Human annotators are then used to rank the generated text outputs from the LM. One may initially think that humans should apply a scalar score directly to each piece of text in order to generate a reward model, but this is difficult to do in practice. Instead, one method that 

## 3.2 - Fine-tuning with RL

as